### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [3]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [4]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model


In [5]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [6]:
# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [7]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


/Users/mkaratadipalayam/mohan/technotes/server/gen-ai/GenAI-Basics/annclassification-try/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mkaratadipalayam/mohan/technotes/server/gen-ai/GenAI-Basics/annclassification-try/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mkaratadipalayam/mohan/technotes/server/gen-ai/GenAI-Basics/annclassification-try/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mkaratadipalayam/mohan/technot

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50


2024-10-30 06:03:34.426964: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-10-30 06:03:34.456977: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-10-30 06:03:34.462126: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-10-30 06:03:34.479283: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-10-30 06:03:34.479338: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-10-30 06:03:34.486856: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-10-30 06:03:34.506957: I tensorflow/core/grappler/optimizers/cust

 32/167 [====>.........................] - ETA: 2s - loss: 0.9265 - accuracy: 0.50006

...

Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Insufficient Memory (00000008:kIOGPUCommandBufferCallbackErrorOutOfMemory)
	<AGXG13XFamilyCommandBuffer: 0x36133ca00>
    label = <none> 
    device = <AGXG13XDevice: 0x120b4b800>
        name = Apple M1 Pro 
    commandQueue = <AGXG13XFamilyCommandQueue: 0x113cc8400>
        label = <none> 
        device = <AGXG13XDevice: 0x120b4b800>
            name = Apple M1 Pro 
    retainedReferences = 1


167/167 [==============================] - 1s 7ms/step - loss: 2.8711 - accuracy: 0.7411
Epoch 47/50
167/167 [==============================] - 1s 7ms/step - loss: 4.1197 - accuracy: 0.7300
Epoch 48/50
167/167 [==============================] - 1s 7ms/step - loss: 3.2266 - accuracy: 0.7385
Epoch 49/50
167/167 [==============================] - 1s 7ms/step - loss: 3.1834 - accuracy: 0.7533
Epoch 50/50
84/84 [==============================] - 1s 7ms/step
Epoch 1/100


/Users/mkaratadipalayam/mohan/technotes/server/gen-ai/GenAI-Basics/annclassification-try/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


167/167 [==============================] - 2s 8ms/step - loss: 0.5890 - accuracy: 0.7133
Epoch 2/100
167/167 [==============================] - 1s 7ms/step - loss: 0.4511 - accuracy: 0.8011
Epoch 3/100
167/167 [==============================] - 1s 6ms/step - loss: 0.4331 - accuracy: 0.8131
Epoch 4/100
167/167 [==============================] - 1s 6ms/step - loss: 0.4310 - accuracy: 0.8101
Epoch 5/100
167/167 [==============================] - 1s 6ms/step - loss: 0.4311 - accuracy: 0.8112
Epoch 6/100
167/167 [==============================] - 1s 6ms/step - loss: 0.4308 - accuracy: 0.8134
Epoch 7/100
167/167 [==============================] - 1s 6ms/step - loss: 0.4314 - accuracy: 0.8104
Epoch 8/100
167/167 [==============================] - 1s 6ms/step - loss: 0.4309 - accuracy: 0.8140
Epoch 9/100
167/167 [==============================] - 1s 6ms/step - loss: 0.4305 - accuracy: 0.8116
Epoch 10/100
167/167 [==============================] - 1s 6ms/step - loss: 0.4306 - accuracy: 0.8110
E

KeyboardInterrupt: 